In [ ]:
### Part 1: embedding a trained generative AI model into a music app

In [ ]:
from app.audio import create_audio_engine

In [ ]:
import urllib.request

In [ ]:
urllib.request.urlretrieve("https://play.forum.ircam.fr/rave-vst-api/get_model/darbouka_onnx", "darbouka_onnx.ts")

In [ ]:
audio_engine = create_audio_engine("test-loop.wav", "darbouka_onnx.ts")

In [ ]:
audio_engine.start()

In [ ]:
audio_engine.loop.set()

In [ ]:
audio_engine.transform.set()

In [ ]:
audio_engine.set_latent_coordinates([0.1, 0.1, 0.5, 0.5])

In [ ]:
audio_engine.transform.clear()

In [ ]:
audio_engine.stop.set()

In [ ]:
audio_engine.stop.clear()

In [ ]:
### Part 2: Training a model 

In [ ]:
import torch
import numpy as np
import copy
from app.control.xy import Model, XYControl, Mode
from torchsummary import summary

In [ ]:
model = Model()

In [ ]:
model.layers = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 4),
        )

In [ ]:
summary(model, (32, 2))

In [ ]:
xy_control = XYControl(audio_engine.set_latent_coordinates, None, model)

In [ ]:
xy_control.load_data()

In [ ]:
X_train, X_test, y_train, y_test = xy_control._prepare_data()

In [ ]:
xy_control._initialize_training_components()

In [ ]:
best_mse = np.inf # init to infinity
history = []

In [ ]:
def train_one_epoch(X_train, y_train):
        running_loss = 0.0
        batch_start = torch.arange(0, len(X_train), xy_control.batch_size)
        xy_control._model.train()

        for start in batch_start:
            X_batch = X_train[start:start+xy_control.batch_size]
            y_batch = y_train[start:start+xy_control.batch_size]

            # Forward pass
            y_pred = xy_control._model(X_batch)
            loss = xy_control.loss_fn(y_pred, y_batch)
            
            # Backward pass and optimization
            xy_control.optimiser.zero_grad()
            if not loss.requires_grad:
                loss.requires_grad_(True)
            loss.backward()
            xy_control.optimiser.step()

            running_loss += loss.item()
        
        return running_loss / len(batch_start)  # Average loss per batch

In [ ]:
for epoch in range(xy_control.n_epochs): # n_epochs = 100
    running_loss = train_one_epoch(X_train, y_train)
    
    # evaluate accuracy at end of each epoch
    mse = xy_control._validate(X_test, y_test)
    history.append(mse)
    
    # Save best model if improved
    if mse < best_mse:
        best_mse = mse
        xy_control._model_weights = copy.deepcopy(xy_control._model.state_dict())

In [ ]:
xy_control._model_trained = True
xy_control._mode =  Mode.control

In [ ]:
xy_control.receive_coordinates([0, 0], [1, 2, 3, 4])